# Run Structures Stored in Neo4j

We support the direct execution of structures stored in Neo4j.

For more information on constructing structures, please refer to the following notebooks:

- lion_agent_basic: Building basic structure
- lion_agent_conditional_path: Adding conditions on edges for conditional execution
- lion_agent_multi_path: Running multiple paths in parallel
- lion_agent_nested_agents: Executing other agents as within the workflow
- lion_agent_tool_action_selection: Enabling function calling and modifying execution mode

## Initiate Neo4j Driver

In [1]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"
database = "neo4j"

from lionagi.integrations.storage.neo4j import Neo4j

neo4j = Neo4j(uri, user, password, database)

In [2]:
from lionagi.core.agent.base_agent import BaseAgent
from lionagi.core.engine.instruction_map_engine import InstructionMapEngine
from lionagi.core.executor.neo4j_executor import Neo4jExecutor

In [3]:
def output_parser(agent):
    output = []
    for branch in agent.executable.branches.values():
        output.append(branch.to_df())
    return output

### Reload lion_agent_conditional_path

In [4]:
structure = Neo4jExecutor(driver=neo4j, structure_name="math")
executor = InstructionMapEngine()
calculator1 = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [5]:
from lionagi.core.message import Instruction

In [6]:
ddd = {'timestamp': '2024-05-19T20:02:50.037414', 
       'ln_id': '4e657ce912b0bcab4c53b884b58eafb1', 
       'instruction': {'sum the absolute values': 'provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|'}}
obj = Instruction.from_obj(ddd)

In [7]:
result_calc = await calculator1.execute(context={"x": -6, "y": 0, "case": 0})

------------------Welcome: system--------------------


system: You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text.

user: {"sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|"}

assistant 4b5df32c47a08110fe0f23f2e24dc84a: {"role": "assistant", "content": ""}

-----------------------------------------------------


user: {"if previous response is positive": "times 2. i.e. *2", "else": "plus 2. i.e. +2"}

assistant 4b5df32c47a08110fe0f23f2e24dc84a: Understood. Please provide the two numbers.

-----------------------------------------------------


### Reload lion_agent_multi_path

In [8]:
structure = Neo4jExecutor(driver=neo4j, structure_name="math_multi")
executor = InstructionMapEngine()
calculator2 = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [9]:
result_calc2 = await calculator2.execute(context={"x": 2, "y": 3})

------------------Welcome: system--------------------


system: You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text.

user: {"diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|"}

user: {"sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|"}

assistant 402230ff65701fd7d79612dcdd5d99a5: {"role": "assistant", "content": ""}

...

user: {"if previous response is positive": "times 2. i.e. *2", "else": "plus 2. i.e. +2"}

assistant 5d6497943a0d2515cc6197a7674faeae: Please provide the two numbers.

-----------------------------------------------------


assistant 402230ff65701fd7d79612dcdd5d99a5: {"sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|"}

-----------------------------------------------------


### Reload lion_agent_nested_agents

In [10]:
structure = Neo4jExecutor(driver=neo4j, structure_name="comedian")
executor = InstructionMapEngine()
comedian = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [11]:
from lionagi.integrations.storage.storage_util import ParseNode
from lionagi.core.action import func_to_tool, DirectiveSelection

In [12]:
action = DirectiveSelection(directive="direct", directive_kwargs={'reason': True, 'score':True})

In [13]:
comedian_result = await comedian.execute()

------------------Welcome: system--------------------


system: As a comedian, you are sarcastically funny

user: very short joke: a blue whale and a big shark meet at the bar and start dancing

assistant 5c4bbaad41dfc88723dedf1777f7183f: And the bartender says, "Great, now I have to mop up the ocean!"

-----------------------------------------------------


...

assistant 5c4bbaad41dfc88723dedf1777f7183f: A blue whale and a big shark meet at the bar and start dancing. The bartender says, "Great, now I have to mop up the ocean!"

-----------------------------------------------------


user: write joke2 again

assistant 5c4bbaad41dfc88723dedf1777f7183f: A blue whale and a big shark meet at the bar and start dancing. And then they stopped, looked at each other, and the whale said, "I guess we really made a splash!"

-----------------------------------------------------


### Reload lion_agent_tool_action_selection

In [14]:
structure = Neo4jExecutor(driver=neo4j, structure_name="tool_mul")
executor = InstructionMapEngine()
calc_mul = BaseAgent(
    structure=structure, executable=executor, output_parser=output_parser
)

In [15]:
question1 = "A school is ordering laptops for its students. If each classroom has 25 students and the school wants to provide a laptop for each student in its 8 classrooms, how many laptops in total does the school need to order?"
question2 = "A bakery sells cupcakes in boxes of 6. If a customer wants to buy 8 boxes, with each person getting one cupcake, how many people can the customer serve cupcake to?"
import json

context = {"Question1": question1, "question2": question2}
context = json.dumps(context)

In [16]:
calc_result = await calc_mul.execute(context)

------------------Welcome: system--------------------


system: you are asked to perform as a function picker and parameter provider

user: Think step by step, understand the following basic math question and provide parameters for function calling.

assistant d1aa72b8033336532fb8f54e81c8c593:


task: 
Follow the prompt and provide the necessary output.
- Additional instruction: Think step by step, understand the following basic math question and provide parameters for function calling.
- Additional context: {"Question1": "A school is ordering laptops for its students. If each classroom has 25 students and the school wants to provide a laptop for each student in its 8 classrooms, how many laptops in total does the school need to order?", "question2": "A bakery sells cupcakes in boxes of 6. If a customer wants to buy 8 boxes, with each person getting one cupcake, how many people can the customer serve cupcake to?"}
- Reason and prepare actions with GIVEN TOOLS ONLY.
- Give a numeric score in [0, 10] and precision of 0 decimal places.



...

score: 
10


tool_schema: 
{'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Perform multiplication on two numbers.', 'parameters': {'type': 'object', 'properties': {'number1': {'type': 'number', 'description': 'First number to multiply.'}, 'number2': {'type': 'number', 'description': 'Second number to multiply.'}}, 'required': ['number1', 'number2']}}}]}


action_response: 
{'action_1': {'function': 'multiply', 'arguments': {'number1': 25, 'number2': 8}, 'output': 200}, 'action_2': {'function': 'multiply', 'arguments': {'number1': 6, 'number2': 8}, 'output': 48}}


action_performed: 
True


-----------------------------------------------------
